In [1]:
import re
import pickle
from heapq import nlargest

import pandas as pd
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import session_info

In [2]:
# Посмотрим версии используемых модулей.
session_info.show()

In [2]:
# Загрузим ранее сохранённые уникальные навазния компаний.
text_unic = pickle.load(open('companies_name_unic.pickle','rb'))

# Загрузим ранее сохранённый tfidfvectorizer.
my_vectorizer = pickle.load(open('vectorizer_2.pickle','rb'))

def sentence_embedding_fl(string, vectorizer):
    tf_vectorizer = vectorizer
    sentence_list = [tf_vectorizer.transform([word]).toarray() for word in string.split()]
        
    result = np.sum(np.array(sentence_list), axis=0)
    if len(result.shape) == 0:
        result = np.zeros((1,9776))
    
    return result

In [3]:
# Загрузим ранее сохранённые эмбеддинги для названий компаний.
name_embeddings = pickle.load(open('name_embeddings.pickle','rb'))

In [4]:
# Тестовый текст 1
my_test_text_1 = 'Darth Vader is a great Sith Lord'

# Получим эмбеддинг тестового текста
text_vector = sentence_embedding_fl(my_test_text_1, my_vectorizer)

In [5]:
# Вычислим косинусное сходство между тестовым текстом и навзаниями компаний.
cosine_similarity_list = [cosine_similarity( text_vector, sentence_vector)[0] for sentence_vector in name_embeddings ]

In [6]:
top = 5

# Функция для поиска топ 5 схожих названий.
def find_top5(similarity_list, top):
    company_names = []
    maximums = nlargest(top, similarity_list)
    indexes_set = set()
    for maximum in maximums:
        indexes = list(np.where(np.array(similarity_list) == maximum)[0])
        for index in indexes:
            if index not in indexes_set:
                indexes_set.add(index)
                company_names.append(text_unic[index])
                
    return  company_names                   


names = find_top5(cosine_similarity_list, top)
N = len(names)

# Так как одно косинусное сходство может быть у нескольких элементов, то список может быть больше 5.
# Поэтому будем выводить все похожие названия, если найдено меньше 5 и только 5, если больше.
if N<=5:
    for name in names:
        print(name)
else:
    for name in names[:5]:
        print(name)

Lord Industrial Ltda
Lord India Pvt., Ltd.
Jm Lord International Llc
Greatech Philippines Inc.
Great Sports Infra Pvt., Ltd.


In [ ]:
# Видим, что к непохожему на название компании тексту нашлись названия, где фигурируют похожие слова "Lord" и "Great".

In [7]:
# Теперь попробуем текст, который был в датасете.
my_test_text_2 = 'Powermax Rubber Factory'

# Получим эмбеддинг второго тестового текста
text_vector2 = sentence_embedding_fl(my_test_text_2, my_vectorizer)

# Вычислим косинусное сходство между тестовым текстом и навзаниями компаний.
cosine_similarity_list2 = [cosine_similarity( text_vector2, sentence_vector)[0] for sentence_vector in name_embeddings ]

names2 = find_top5(cosine_similarity_list2, top)
N2 = len(names2)

# Так как одно косинусное сходство может быть у нескольких элементов, то список может быть больше 5.
# Поэтому будем выводить все похожие названия, если найдено меньше 5 и только 5, если больше.
if N2<=5:
    for name in names2:
        print(name)
else:
    for name in names2[:5]:
        print(name)

Powermax Rubber Factory
B G N Rubber Factory
Western Rubbers India Pvt., Ltd.
K.S. Rubbers
Imp. Rubbers International Llc


In [ ]:
# Видим что на первом месте исходный текст (не удивительно), а дальше тексты с похожими словами. 

In [8]:
# Теперь попробуем текст, не был в датасете, но похож, на то, что был там.
# Оригинал: Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
# Изменённый вариант
my_test_text_3 = 'Well Aero System Lab Industrial Craft'

# Получим эмбеддинг второго тестового текста
text_vector3 = sentence_embedding_fl(my_test_text_3, my_vectorizer)

# Вычислим косинусное сходство между тестовым текстом и навзаниями компаний.
cosine_similarity_list3 = [cosine_similarity( text_vector3, sentence_vector)[0] for sentence_vector in name_embeddings ]

names3 = find_top5(cosine_similarity_list3, top)
N3 = len(names3)

# Так как одно косинусное сходство может быть у нескольких элементов, то список может быть больше 5.
# Поэтому будем выводить все похожие названия, если найдено меньше 5 и только 5, если больше.
if N3<=5:
    for name in names3:
        print(name)
else:
    for name in names3[:5]:
        print(name)

Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
Aerocosta Global Systems Inc.
Craft Colombia Sas
Mx Systems International P Ltd.
Craft Argentina S.A.


In [ ]:
# Видим что на первом месте текст, который был изменён а дальше тексты с похожими словами. 